In [1]:
# package loading
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime, timedelta

# Snowflake Cloud 접속 정보
snowflake_account = 'ls58031.ap-northeast-2.privatelink'
snowflake_user='21600594'
snowflake_password='test123!A'
snowflake_database='HKT_DW'
snowflake_warehouse='SMALL_WH'
snowflake_schema = ''

# Oracle DB 접속 정보
oracle_user = 'hqgmes'
oracle_password = 'hqgmes'
oracle_dsn = '202.31.25.83:1521/DKPPODA.kppodad'

# Oracle Clinet 설치 결로
oracle_dir = r"C:\Users\hanta\Downloads\instantclient_21_10"

import snowflake.connector
import oracledb
oracledb.init_oracle_client(lib_dir=oracle_dir)

# Table reading fun.
def db_table(quary, db_type):
    # snowflake 데이터 불러오기
    if db_type == 'sf' :
        con = snowflake.connector.connect(
            account = snowflake_account,
            user = snowflake_user,
            password = snowflake_password,
            database = snowflake_database,
            warehouse = snowflake_warehouse
            )
        cur = con.cursor()
        cur.execute(quary)
        result_df = cur.fetch_pandas_all()
        con.close
        return result_df
    
    # Oracle DB 데이터 불러오기
    elif db_type == 'oda':
        con = oracledb.connect(
            user = oracle_user, 
            password = oracle_password, 
            dsn = oracle_dsn
            )
        cur = con.cursor()
        cur.execute(quary)
        table_cols = [cols[0] for cols in cur.description]
        result_df = pd.DataFrame(cur.fetchall(), columns=table_cols)
        con.close
        return result_df 
    else:
        print('oda 또는 sf을 변수로 입력하세요')


In [2]:
# Quary 변수에 할당
ref_date1 = 20230501 # 2023년 01월
ref_date2 = 20230630 # 2023년 12월

# 15일 전/후의 날짜 계산
date_obj = datetime.strptime( str(ref_date1), '%Y%m%d')
date_15_days_ago = date_obj - timedelta(days=15)

date_obj2 = datetime.strptime( str(ref_date2), '%Y%m%d')
date_15_days_later = date_obj2 + timedelta(days=15)

pre_ref_date1 = int(date_15_days_ago.strftime('%Y%m%d'))
after_ref_date2 = int(date_15_days_later.strftime('%Y%m%d'))


# MES 기준 : LWD307 - 정련 운전 실적 / RUNSUMMARY
LQLTTR164 = f'''   
    SELECT 
        PLT_CD
        ,MC_CD
        ,LOT_NO
        ,WRK_ORD_NO
        ,ORD_SEQ
        ,SPEC_CD
        ,SPEC_VER
        ,MCS_VER
        ,MC_RUN_MODE
        ,PRDT_DATE
        ,MAX(BTCH_NO) AS BTCH_NO
        ,MIN(WRK_SRT_DTM) AS WRK_SRT_DTM
        ,MAX(WRK_END_DTM) AS WRK_END_DTM
    FROM DPPMES.LQLTTR164
    WHERE 1=1
        AND LOT_NO BETWEEN '{ref_date1}' AND '{ref_date2}'
        -- AND LOT_NO = 202301040002
        -- AND SPEC_CD = 'C1C49S019'
        -- AND MC_CD = 'A1002'
        -- ORDER BY 1,2,3,4,5,6
    GROUP BY   
        PLT_CD
        ,MC_CD
        ,LOT_NO
        ,WRK_ORD_NO
        ,ORD_SEQ
        ,SPEC_CD
        ,SPEC_VER
        ,MCS_VER
        ,MC_RUN_MODE
        ,PRDT_DATE
'''

# MES 기준 : LWD333 - 컴파운드 생산별 투입 원부재, 고무
LQLTTR147 = f'''
    SELECT 
        PLT_CD
        ,MC_CD
        ,LOT_NO
        ,BTCH_NO
        ,RMTL_TP_CD 
        ,RMTL_BIN_NO 
        ,RMTL_IPT_DGE
        ,RMTL_CD
        ,SPEC_CD
        ,SPEC_VER
        ,MCS_VER
        ,PRDT_DTM
        ,UDT_DTM
        ,MRM_WGT
    FROM DPPMES.LQLTTR147
    WHERE 1=1
        AND LOT_NO BETWEEN '{ref_date1}' AND '{ref_date2}'
'''

# MES 기준 : LWD841 - 원부재 투입 실적
LQLTTR151 = f'''
    SELECT 
        PLT_CD
        ,SCAN_DTM -- TIMIESTAMPE로 변경
        ,MC_CD
        -- ,SCAN_RMTL_CD
        ,PRDT_DATE
        ,RMTL_TP_CD
        ,RMTL_CD
        ,LOT_NO
        ,SPEC_CD
        ,SPEC_VER
        ,MCS_VER
        ,WRK_FG
        ,WRK_FG_NM
        ,SCAN_BARCD_NO
        ,WHS_LOT_NO
        ,CMP_CD
        ,CMP_NM    
    FROM DPPMES.LQLTTR151
    WHERE SCAN_DTM BETWEEN '{pre_ref_date1}' AND '{ref_date2}'
'''

# ODA TRXX_TRACE 데이터
TRXX_TRACE = f'''
    SELECT *
    FROM BI_XV_TRXX_TRACE_SETA
    WHERE PLN_DATE BETWEEN '{ref_date1}' AND '{ref_date2}'
'''

# MES기준 LQE060 : RPA 검사 데이터 
LQLTTR268 = f'''
    SELECT * 
    FROM DPPMES.LQLTTR268
    WHERE MXG_PRDT_DATE BETWEEN '{ref_date1}' AND '{ref_date2}'
'''

# MES기준 LQE431 : BATCH 검사 데이터
LQLTTR171 = f'''
    SELECT *
    FROM DPPMES.LQLTTR171
    WHERE MXG_PRDT_DATE BETWEEN '{ref_date1}' AND '{ref_date2}'
'''

# MES기준 LQE461 : 고무물성(정밀) 검사 데이터
LQLTTR177 = f'''
    SELECT *
    FROM DPPMES.LQLTTR177
    WHERE MXG_PRDT_DATE BETWEEN '{ref_date1}' AND '{ref_date2}'
'''


# ODA TRXX_TRACE 데이터
TRXX_TRACE = f'''
    SELECT *
    FROM BI_XV_TRXX_TRACE_SETA
    WHERE PLN_DATE BETWEEN '{ref_date1}' AND '{ref_date2}'
    AND PLT_CD = 'DP'
'''

In [3]:
# 테이블 할당 완료
df_lwd333_lqlttr147 = db_table(LQLTTR147,'sf')
df_lwd307_lqlttr164 = db_table(LQLTTR164,'sf')
df_lwd841_lqlttr151 = db_table(LQLTTR151,'sf')
df_trxx_trace = db_table(TRXX_TRACE,'oda')

df_lqe060_lqlttr268 = db_table(LQLTTR268,'sf')
df_lqe431_lqlttr171 = db_table(LQLTTR171,'sf')
df_lqe461_lqlttr177 = db_table(LQLTTR177,'sf')

# GLP 출고 데이터(추출, ODA I/F 진행 예정) → 원료 투입 데이터 형태로 입력(Refer. LWD841)
dfrelease_path = r"C:\Users\hanta\OneDrive - HKNC\Arena Messenger Download\6. 출고이력 RAW_WM_P_L2141_2023-09-20 (2).csv"
dfrelease = pd.read_csv(dfrelease_path)
dfrelease.rename (columns = {'생성일자':'CREDAT' , 
                             '생성일시':'CREDATIM', 
                             'Material':'SKUKEY', 
                             'LOT':'LOTA02', 
                             "업체":"LOTA01", 
                             "W/H":"SWERKS"} , inplace = True)
dfrelease['CREDATIM'] = dfrelease['CREDATIM'].astype(str)
## 생성일자 = CREDAT
## 생성일시 = CREDAT + CRETIM
## Material = SKUKEY
## LOT = LOTA02
## 업체 = LOTA01
## W/H = SWERKS

# Drop 처리
df_trxx_trace = df_trxx_trace.dropna(axis=1, how='all')
df_lqe060_lqlttr268 = df_lqe060_lqlttr268.dropna(axis = 1, how = 'all')
df_lqe431_lqlttr171 = df_lqe431_lqlttr171.dropna (axis = 1, how = 'all')
df_lqe461_lqlttr177 = df_lqe461_lqlttr177.dropna(axis = 1, how = 'all')

# DTM 처리
df_lwd841_lqlttr151['SCAN_DTM'] = pd.to_datetime(df_lwd841_lqlttr151['SCAN_DTM'])
df_lwd333_lqlttr147['UDT_DTM'] = pd.to_datetime(df_lwd333_lqlttr147['UDT_DTM'])
dfrelease['CREDATIM'] = pd.to_datetime(dfrelease['CREDATIM'])


# 원료 정련 투입 이력 →  강제로 LWD841꼴로 만들기 (설비란은 비워둠)
df_lwd841_lqlttr151_cols =list(df_lwd841_lqlttr151.columns)
dfrelease_len = len(dfrelease)
df_lwd841_lqlttr151_release = pd.DataFrame(index=range(0,dfrelease_len), columns = df_lwd841_lqlttr151_cols)
df_lwd841_lqlttr151_release['PLT_CD'] =  'DP'
df_lwd841_lqlttr151_release['SCAN_DTM'] =  dfrelease['CREDATIM']
df_lwd841_lqlttr151_release['CMP_NM'] = dfrelease['업체명']
df_lwd841_lqlttr151_release['RMTL_CD'] = dfrelease['SKUKEY']
df_lwd841_lqlttr151_release['WHS_LOT_NO'] = dfrelease['LOTA02']
df_lwd841_lqlttr151_release['CMP_CD']= dfrelease['LOTA01']
# 얘네는 정말 정련 출고 데이터라, 어느 설비 (믹싱기)로 들어가는 지 정보가 없음
# 그래서 날짜 기준으로 넣을 예정 (같은 Material의 다른 LOT 출고 전까지, 이전 LOT 를 컴파운드에 넣는다는 가정)
# 이미 매칭된 애들 (바코드 정보 남아있는 것) 제외하고 컴파운드에 매칭할 것

In [6]:
print(f"""
{df_lwd333_lqlttr147.shape}
{df_lwd307_lqlttr164.shape}
{df_lwd841_lqlttr151.shape}
{df_trxx_trace.shape}

{df_lqe060_lqlttr268.shape}
{df_lqe431_lqlttr171.shape}
{df_lqe461_lqlttr177.shape}

{df_trxx_trace.shape}
{df_lqe060_lqlttr268.shape}
{df_lqe431_lqlttr171.shape}
{df_lqe461_lqlttr177.shape}
"""
)


(1392595, 14)
(6922, 13)
(19601, 16)
(3446, 38)

(10160, 66)
(128582, 63)
(2746, 90)

(3446, 38)
(10160, 66)
(128582, 63)
(2746, 90)



In [9]:
### 컴파운드 생산에 들어간 전체 원료-LOT 연결 (LWD333+LWD841)
# 한 믹싱런에서는 같은 원료 들어갔다고 가정하고, PLT, MC, SPEC일치하면 RMTL 유니크하게 남겨두기
df_lwd333_lqlttr147_uniqueRM = df_lwd333_lqlttr147.drop_duplicates(subset=['PLT_CD', 'LOT_NO','MC_CD', 'SPEC_CD', 'RMTL_CD'], keep='first')


# 설비 비어있는 것 (정련출고 only) Merge
release_col = ['PLT_CD', 'SCAN_DTM', 'PRDT_DATE', 'RMTL_CD','WHS_LOT_NO', 'CMP_CD', 'CMP_NM' ]
merged_run_release = pd.merge(df_lwd333_lqlttr147_uniqueRM, df_lwd841_lqlttr151_release[release_col], on=['PLT_CD','RMTL_CD'],how='left')

## 투입 일자가 정련 시작보다 직전인 것 중에 가장 큰 값만 남기기
filtered_df = merged_run_release[merged_run_release['SCAN_DTM'] <= merged_run_release['UDT_DTM']]
max_d_row = filtered_df.groupby(['LOT_NO', 'SPEC_CD', 'RMTL_CD'])['SCAN_DTM'].idxmax()
filtered_merged_run_release = filtered_df.loc[max_d_row]


# 설비 있는 거 Merge
df_lwd841_lqlttr151 = df_lwd841_lqlttr151.drop ( df_lwd841_lqlttr151[df_lwd841_lqlttr151['WHS_LOT_NO']==' '].index)
lqlttr151_cols = ['PLT_CD', 'SCAN_DTM', 'MC_CD', 'PRDT_DATE', 'RMTL_CD','WHS_LOT_NO', 'CMP_CD', 'CMP_NM' ]
merged_run_input = pd.merge(df_lwd333_lqlttr147_uniqueRM, df_lwd841_lqlttr151[lqlttr151_cols], on=['PLT_CD','RMTL_CD', 'MC_CD', ],how='inner')

## 투입 일자가 정련 시작보다 직전인 것 중에 가장 큰 값만 남기기
filtered_df_2 = merged_run_input[merged_run_input['SCAN_DTM'] <= merged_run_input['UDT_DTM']]
max_d_row_2 = filtered_df_2.groupby(['MC_CD', 'LOT_NO', 'SPEC_CD', 'RMTL_CD'])['SCAN_DTM'].idxmax()
filtered_merged_run_input = filtered_df_2.loc[max_d_row_2]

# 최종 합치기 - 생산LOT 에 들어간 원료 연결한 최종본
raw_mixing_fin = pd.concat([filtered_merged_run_release, filtered_merged_run_input], axis=0).drop_duplicates()
raw_mixing_fin['SHORT_SPEC_CD'] = raw_mixing_fin['SPEC_CD'].str[2:]


### 동일 CQ LOT에 들어가는 것 다 합치기
### 재료 바코드로 변경
pd.set_option('mode.chained_assignment',  None) # 경고 off

# trace 데이터 한 줄씩 돌아가면서 한 CQ 컴파운드+믹싱런에 들어간 C1, C2, C2 로뜨 리스트로 만들기
df_trxx_trace['SHORT_SPEC_CD'] = df_trxx_trace['SPEC_CD'].str[2:]
df_trxx_trace_filtered = df_trxx_trace[ df_trxx_trace['SPEC_CD'].str[:2] == 'CQ']

trace_input_mat = []
for i, row in df_trxx_trace_filtered.iterrows():
    shtcompd = row['SHORT_SPEC_CD']
    lotnum = row['LOT_NO']
    mccd = row['MC_CD']
    
  # trace CQ 믹싱런 한개로 모아지 C1, C2, C3단계의 로뜨넘버 모으기  ①
    step_lot = row[['LOT_NO', 'L1_LOT_NO', 'L2_LOT_NO', 'L3_LOT_NO', 'L4_LOT_NO', 'L5_LOT_NO']].tolist()
    step_lot = np.array([str(x) for x in step_lot if x != None])

  # trace CQ 믹싱런 한개에 있는 C1, C2, C3단계의 믹싱기 모으기  ②
    mixer =  row[['MC_CD','L1_MC_CD', 'L2_MC_CD', 'L3_MC_CD', 'L4_MC_CD', 'L5_MC_CD']].tolist()
    mixer = np.array([str(x) for x in mixer if x != None])


  # 컴파운드 명이 같고, 믹싱런과 설비가 위에  ①, ② 리스트에 포함된다면, 해당 행 맨 오른쪽에 trace CQ 믹싱런, 설비 적기
  # (그 행의 원료가 최종적으로 들어가는 CQ의 믹싱런넘버)
    c1 = raw_mixing_fin['SHORT_SPEC_CD'] == shtcompd
    c2 = raw_mixing_fin['LOT_NO'].isin(step_lot)  
    c3 = raw_mixing_fin['MC_CD'].isin(mixer) 
    condition = c1&c2&c3

    input_mat = raw_mixing_fin[condition]

    input_mat['FIN_LOT_NO']=len(input_mat) * [lotnum]
    input_mat['FIN_MC_CD']=len(input_mat) * [mccd]

    if not input_mat.empty:   trace_input_mat.append(input_mat)

trace_raw_runsum = pd.concat(trace_input_mat, ignore_index=True).drop_duplicates()

# RPA & Batch 연결
df_rpa_batch = pd.merge( df_lqe431_lqlttr171, df_lqe060_lqlttr268 , on=['PLT_CD', 'LOT_NO', 'SPEC_CD', 'MXG_MC_CD', 'BTCH_NO'], how='left')
df_rpa_batch.rename(columns = {'MXG_PRDT_DATE_x' : 'MXG_PRDT_DATE'}, inplace = True)

# 고무물성(BIS,MV, Rheo)결과 추가 연결
df_rpa_batch_property = pd.merge(df_rpa_batch, df_lqe461_lqlttr177, on=['MXG_PRDT_DATE', 'PLT_CD', 'SPEC_CD', 'MXG_MC_CD', 'BTCH_NO'], how='left')
df_rpa_batch_property['SHORT_SPEC_CD'] = df_rpa_batch_property['SPEC_CD'].str[2:]

df_rpa_batch_property = df_rpa_batch_property.dropna (axis = 1, how='all')
df_rpa_batch_property = df_rpa_batch_property.drop_duplicates()

# 모든 배치, RPA 검사, 고무물성  -  업체 / LOT 연결하는 작업
trace_raw_runsum.drop(labels = ['UDT_DTM','SPEC_VER'], axis = 1, inplace = True)
df_final = pd.merge( df_rpa_batch_property, trace_raw_runsum , left_on=['PLT_CD','MXG_MC_CD', 'MXG_LOT_NO', 'SPEC_CD'], right_on=['PLT_CD', 'FIN_MC_CD','FIN_LOT_NO', 'SPEC_CD'], how='left')
df_final = df_final.drop_duplicates()

# 모든 행이 빈 열 삭제
print(df_final.shape)
df_final = df_final.dropna(axis = 1, how = 'all')

# 원료정보 + CQ 로뜨정보 없는 행 삭제
df_final = df_final.dropna(subset=['CMP_NM'])
df_final = df_final.dropna(subset=['FIN_LOT_NO'])

df_final.shape

(364419, 228)


(307965, 228)